In [1]:
import os
import csv
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [3]:
train_bodies = pd.read_csv('/content/drive/MyDrive/train_bodies.csv')
#train_bodies.head(10)

test_bodies = pd.read_csv('/content/drive/MyDrive/competition_test_bodies.csv')
#test_bodies.head(10)

# load training data
train_df = pd.read_csv('/content/drive/MyDrive/train_stances.csv')
# load testing data
test_df = pd.read_csv('/content/drive/MyDrive/competition_test_stances.csv')

train_df.replace('agree',0,True)
train_df.replace('disagree',1,True)
train_df.replace('discuss',2,True)
train_df.replace('unrelated',3,True)

test_df.replace('agree',0,True)
test_df.replace('disagree',1,True)
test_df.replace('discuss',2,True)
test_df.replace('unrelated',3,True)

combined_df_train = train_df.join(train_bodies.set_index('Body ID'), on='Body ID')
combined_df_test = test_df.join(test_bodies.set_index('Body ID'), on='Body ID')

headlines = list()

for i in combined_df_train['Headline']:
    if i not in headlines:
        headlines.append(i)

temp2 = combined_df_train.loc[combined_df_train['Stance'] == 3]

df = pd.DataFrame()
cnt, sum1 = 0, 0

for i in headlines:
    temp1 = combined_df_train.loc[combined_df_train['Headline'] == i]
    related = temp1.loc[(temp1['Stance'] == 0) | (temp1['Stance'] == 1) | (temp1['Stance'] == 2)]
    df_t = df.append(related)
    df = df_t
    unrelated = temp1.loc[temp1['Stance'] == 3]
    if related.shape[0] < 1:
        cnt = cnt + 1
        sum1 = sum1 + unrelated.shape[0]
        continue
    if unrelated.shape[0] < related.shape[0]:
        df_t = df.append(unrelated)
    else:
        df_t = df.append(unrelated.iloc[:int(related.shape[0])])
    df = df_t
  
#df_t = df.append(df1)
#df = df_t.append(df2.iloc[:df1.shape[0]])

temp2 = df[df['Stance'] == 3]

stances = df['Stance']
stances.replace(0, 'agree',True)
stances.replace(1, 'disagree',True)
stances.replace(2, 'discuss',True)
stances.replace(3, 'unrelated',True)

df['Stance'] = stances

df = df.sample(frac = 1)

df.to_csv('full_train.csv', index=False, encoding='utf-8')

stances1 = combined_df_test['Stance']
stances1.replace(0, 'agree',True)
stances1.replace(1, 'disagree',True)
stances1.replace(2, 'discuss',True)
stances1.replace(3, 'unrelated',True)
combined_df_test['Stance'] = stances1

combined_df_test.to_csv('full_test.csv', index=False, encoding='utf-8')

In [4]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 27.3 MB/s 
     |████████████████████████████████| 1.2 MB 63.6 MB/s 
     |████████████████████████████████| 1.8 MB 56.4 MB/s 
     |████████████████████████████████| 6.6 MB 64.7 MB/s 
     |████████████████████████████████| 4.7 MB 38.9 MB/s 
     |████████████████████████████████| 9.1 MB 58.1 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 365 kB 71.4 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 74.3 MB/s 
     |████████████████████████████████| 181 kB 76.9 MB/s 
     |████████████████████████████████| 157 kB 59.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 156 kB 68.3 MB/s 
     |████████████████████████████████| 141 kB 77.6 MB/s 
     |█████████████████████████████

In [5]:
# https://stackoverflow.com/questions/56850711/importerror-please-install-apex-from-https-www-github-com-nvidia-apex-to-use

import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

USE_APEX = True

"""if USE_APEX:
            with timer('install Nvidia apex'):
                # Installing Nvidia Apex
                os.system('git clone https://github.com/NVIDIA/apex; cd apex; pip install -v --no-cache-dir' + 
                          ' --global-option="--cpp_ext" --global-option="--cuda_ext" ./')
                os.system('rm -rf apex/.git') # too many files, Kaggle fails
                from apex import amp"""

'if USE_APEX:\n            with timer(\'install Nvidia apex\'):\n                # Installing Nvidia Apex\n                os.system(\'git clone https://github.com/NVIDIA/apex; cd apex; pip install -v --no-cache-dir\' + \n                          \' --global-option="--cpp_ext" --global-option="--cuda_ext" ./\')\n                os.system(\'rm -rf apex/.git\') # too many files, Kaggle fails\n                from apex import amp'

In [6]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [7]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 10069, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 10069 (delta 89), reused 125 (delta 57), pack-reused 9884
Receiving objects: 100% (10069/10069), 14.94 MiB | 13.31 MiB/s, done.
Resolving deltas: 100% (6893/6893), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/pytho

In [8]:
from simpletransformers.classification import ClassificationModel

In [9]:
import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

In [10]:
import pandas as pd
import numpy as np

In [11]:

train_raw = pd.read_csv('/content/drive/MyDrive/full_train.csv')
test_raw = pd.read_csv('/content/drive/MyDrive/full_test.csv')

In [12]:
train = pd.DataFrame(columns=['text_a', 'text_b', 'labels'])
train['text_a'] = train_raw['Headline']
train['text_b'] = train_raw['articleBody']

In [13]:
stances = train_raw['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances
train['labels'] = stances

In [14]:
stances = test_raw['Stance']
stances.replace('agree', 0,True)
stances.replace('disagree', 1, True)
stances.replace('discuss', 2, True)
stances.replace('unrelated', 3, True)
stances

0        3
1        3
2        3
3        3
4        3
        ..
25408    0
25409    2
25410    1
25411    1
25412    0
Name: Stance, Length: 25413, dtype: int64

In [15]:
test = pd.DataFrame(columns=['text_a', 'text_b'])

test['text_a'] = test_raw['Headline']
test['text_b'] = test_raw['articleBody']
#test['labels'] = test_raw['label']

In [16]:
test['labels'] = stances

In [17]:
model = ClassificationModel('roberta', 'roberta-base',  num_labels=4,args={
    'learning_rate':1e-5,
    'num_train_epochs': 1,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'evaluate_during_training_steps': False,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'fp16': True,
    'early_stopping': True
})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [18]:
model.train_model(train)

  0%|          | 0/26854 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/6714 [00:00<?, ?it/s]

(6714, 0.39877766591383296)

In [19]:
test_ll = list()

for i in range(len(test)):
  test_ll.append([test['text_a'][i], test['text_b'][i]])

In [20]:
def acc(pred):
  cnt = 0
  for i in range(len(pred)):
    if pred[i] == stances[i]:
      cnt = cnt + 1
  print(cnt / len(pred))

In [21]:
def createAns(opt, epoch):
  df_answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
  df_answer['Stance'] = opt
  df_answer.replace(0, 'agree',True)
  df_answer.replace(1, 'disagree',True)
  df_answer.replace(2, 'discuss',True)
  df_answer.replace(3, 'unrelated',True)
  df_answer['Headline'] = test_raw['Headline']
  df_answer['Body ID'] = test_raw['Body ID']
  fName = 'answerEp' + str(epoch) + '.csv'
  df_answer.to_csv(fName, index=False, encoding='utf-8')
  return df_answer

In [22]:
model = ClassificationModel(
    "roberta", "outputs/checkpoint-6714-epoch-1"
)

In [23]:
predictions, raw_outputs = model.predict(test_ll)

  0%|          | 0/25413 [00:00<?, ?it/s]

  0%|          | 0/6354 [00:00<?, ?it/s]

In [24]:
acc(predictions)

0.9318852555778538


In [25]:
createAns(predictions, 5)

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,agree
25409,The success of the Affordable Care Act is a hu...,2583,discuss
25410,The success of the Affordable Care Act is a hu...,2584,discuss
25411,The success of the Affordable Care Act is a hu...,2585,agree


In [26]:
df_answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])

In [27]:
df_answer['Stance'] = predictions

In [28]:
df_answer.replace(0, 'agree',True)
df_answer.replace(1, 'disagree',True)
df_answer.replace(2, 'discuss',True)
df_answer.replace(3, 'unrelated',True)

In [29]:
df_answer['Headline'] = test_raw['Headline']
df_answer['Body ID'] = test_raw['Body ID']

In [30]:
df_answer.to_csv('answer.csv', index=False, encoding='utf-8')